University of Michigan - ROB 201 Calculus for the Modern Engineer

---

# Julia HW03: Chapters 3.1.2-3.3.4 

### Topics Covered
- Riemann Lower-Upper Sums vs Closed-form Solutions
- Numerical Methods
  - Trapezoidal Rule
  - Error Estimation
  - Simpson's Rule
- Generalized Additivity of the Riemann Integral

<br>

<p align="center">
  <img src="data/HW03SurfingWaveSmall.png", width="70%">
</p>

> The integral of speed is position! In a car, the faster you go, as reported by the speedometer, the farther you travel, as reported by the odometer. Surfing, if your current wave is triple the speed of your previous wave, you'll travel three times farther in the same amount of time. Makes sense?

# Problem 1: Exact Integrals for Monomials and Polynomials

## P.1A: Monomials
Write a function that computes the exact value of $\int_a^b x^k dx$ where $k\ge 0$ is an integer. Your function should work for both $a \ge b$ and $a < b$.

**Hint:** Read Chapter 3 on integrating monomials.

In [ ]:
# Build your function here by filling in any missing lines

function exactMonomialIntegrator(k, a, b)
    # k = degree of monomial
    # a = lower limit and b = upper limit
    if k < 0
        k = 0
    end
    k = floor(Int64, k) # make sure k is an integer

    ###
    ### YOUR CODE HERE
    ###

    return intAtoB
end

In [ ]:
#= Friendly Check =#
# if the value of is_it_correct_checkN is "Yes", then your answer is LIKELY correct. 
# If the value of is_it_correct_checkN is "No", then your answer is DEFINITELY wrong
#
intAtoB1 = exactMonomialIntegrator(5, -4.0, 3.0)
intAtoB2 = exactMonomialIntegrator(3, -2, 8.0)

# Run Tests
is_it_correct_check1a_1 = abs(intAtoB1 + 561.1666666666666) < 1e-6 ? "Yes" : "No"
is_it_correct_check1a_2 = abs(intAtoB2 - 1020.0) < 1e-6 ? "Yes" : "No"

@show is_it_correct_check1a_1;
@show is_it_correct_check1a_2;

## P.1B: Polynomials

Write a function that computes the exact value of 
$$ \int_a^b \left( \alpha_n x^n + \alpha_{n-1} x^{n-1} + \cdots + \alpha_1 x + \alpha_0\right) dx $$ 

where $n \ge 0$ is an integer, and $a$ and $b$ are finite real numbers. Your function should work for both $a \ge b$ and $a < b$.

**Hint:** Read Chapter 3 on integrals of linear combinations of functions.

**Note:** You can assume that polyCoeff[k] = $\alpha_{n+1-k}$ for $ 1 \le k \le (n+1)$. Recall that Julia is 1-indexed. Hence, the first entry of polyCoeff is $\alpha_n$ which goes with $x^n$ and the last entry is $\alpha_0$ which goes with $x^0$. 

In [ ]:
# Build your function here by filling in any missing lines

function exactPolynomialIntegrator(polyCoeff, a, b)
    # polyCoeff = [alpha_n; alpha_{n-1}; ...; alpha_1; alpha_0]
    # a = lower limit and b = upper limit
    n = length(polyCoeff) - 1
    if n < 0
        return NaN
    end
    intAtoB = 0.0

    ###
    ### YOUR CODE HERE
    ###

    return intAtoB
end

In [ ]:
#= Friendly Check =#
# if the value of is_it_correct_checkN is "Yes", then your answer is LIKELY correct. 
# If the value of is_it_correct_checkN is "No", then your answer is DEFINITELY wrong
#
polyCoeff = [3 2 1]
a, b = (0, 1)
intAtoB01 = exactPolynomialIntegrator(polyCoeff, a, b)
if abs(intAtoB01 - 4.333333333333333) < 1e-3
    println("You are interpreting the vector polyCoeff backward: the first 
             entry goes with x^n and not x^0. \n")
end
# Run Tests
is_it_correct_check1b = abs(intAtoB01 - 3.0) < 1e-6 ? "Yes" : "No"
@show is_it_correct_check1b;

In [ ]:
# ========== PROBLEM 1 SAMPLE SOLUTION ==========
# You can use this as a reference to check your answer.

### 1A BEGIN SOLUTION
function exactMonomialIntegrator(k, a, b)
    if k < 0
        k = 0
    end
    k = floor(Int64, k)
    intAtoB = (b^(k + 1) - a^(k + 1)) / (k + 1.0)
    return intAtoB
end
### 1A END SOLUTION

### 1B BEGIN SOLUTION
function exactPolynomialIntegrator(polyCoeff, a, b)
    n = length(polyCoeff) - 1
    if n < 0
        return NaN
    end
    intAtoB = 0.0
    for k = 1:(n+1)
        temp = exactMonomialIntegrator(n + 1 - k, a, b)
        intAtoB += polyCoeff[k] * temp
    end
    return intAtoB
end
### 1B END SOLUTION

In [ ]:
# ====================================================
# 🔒 GRADER CELL — Problem 1 (Do Not Edit)
# ====================================================
# This cell required previous Friendly checks to be run first.
# Please run this cell before going to the next problem to avoid variable errors.

point_1A = is_it_correct_check1a_1 == "Yes" && is_it_correct_check1a_2 == "Yes" && abs(exactMonomialIntegrator(0, -5, 5) - 10) < 1e-2 ? 1 : 0
point_1B = is_it_correct_check1b == "Yes" && abs(exactPolynomialIntegrator([1, 2, 3], 0, 1) - 4.33333) < 1e-2 ? 1 : 0
total_score_1 = point_1A + point_1B

# Show score
println("Problem 1 Score: $total_score_1 / 2")

# Problem 2: Comparing Standard Riemann Integration Based on Rectangles to Exact Values for Polynomials

### Notes
```julia
function myRiemannIntegralValueAndError(f, a, b, N, aTol=1e-6)
    # Works for b < a 
    if abs(b-a) < aTol
        println("a and b are too close together for most numerical integration methods.")
        return NaN
    end
    N = floor(Int64, N)
    if N < 2
        N = 2
    end
    Δx = (b-a)/N
#     # You can uncomment these messages if you want to see them
#     if b < a
#         println("Integrating backward with Δx = $Δx")
#     else
#         println("Integrating forward with Δx = $Δx")
#     end
    x = range(a, b, N+1)
    y = f.(x)
    upperSum = 0.0
    lowerSum = 0.0
    for i = 1:N
        lowerSum += Δx *minimum([y[i], y[i+1]])
        upperSum += Δx *maximum([y[i], y[i+1]])
    end
    if b < a # This ensures the upperSum is larger or equal to the lowerSum
             # Could remove and use instead
             # myPMerror =  abs(upperSum - lowerSum)/2.0
        temp = lowerSum
        lowerSum = upperSum
        upperSum = temp
    end    
    myEstimatedIntegral = (upperSum + lowerSum)/2.0
    myPMerror =  (upperSum - lowerSum)/2.0
    return myEstimatedIntegral, myPMerror
end
```
  - Combines several functions from JuliaHW02 into one function.
  - Works for $\Delta x:= \frac{b-a}{N} < 0$ as well as for positive  $\Delta x$. 

  <br>
  
**Run the next two cells.**

In [ ]:
# Test01 Run Me
include("data/basicRiemannIntegrator.jl")
# Pulls in the function myRiemannIntegralValueAndError

f(x) = 4x^2 + 7
polyCoeff = [4; 0.0; 7.0]
a, b = (0, 4)

for k = 4:10
    N = 2^k
    myEstimatedIntegral01, myPMerror01 = myRiemannIntegralValueAndError(f, a, b, N)
    println(" ")
    myExactIntegral01 = exactPolynomialIntegrator(polyCoeff, a, b)
    myExactError01 = myEstimatedIntegral01 - myExactIntegral01
    println("\n N = $N,  Exact Vlaue = $myExactIntegral01, Estimated Integral = $myEstimatedIntegral01
           Estimated Error = $myPMerror01,   True Error $myExactError01")
    println(" ")
end

In [ ]:
# Test02 Run Me
include("data/basicRiemannIntegrator.jl")
# Pulls in the function myRiemannIntegralValueAndError

g(x) = -2 * x^7 + x^4 - 2x^2 - 3x - 4
polyCoeff = [-2.0; 0.0; 0.0; 1; 0.0; -2.0; -3.0; -4.0]
a, b = (0, 4)

for k = 4:10
    N = 2^k
    myEstimatedIntegral02, myPMerror02 = myRiemannIntegralValueAndError(g, a, b, N)
    println(" ")
    myExactIntegral02 = exactPolynomialIntegrator(polyCoeff, a, b)
    myExactError02 = myEstimatedIntegral02 - myExactIntegral02
    println("\n N = $N,  Exact Vlaue = $myExactIntegral02, Estimated Integral = $myEstimatedIntegral02
           Estimated Error = $myPMerror02,   True Error $myExactError02")
    println(" ")
end

## True or False: Determine the following based on the trends of Test01 and Test02

**Q1**: The ESTIMATED ERROR of Riemann Integration with N-rectangles roughly HALVES (i.e., divides by TWO) as $N$ DOUBLES.

**Q2**: The TRUE ERROR of Riemann Integration with N-rectangles roughly HALVES as $N$ DOUBLES.

### Instructions
Answer with upper case "T" for true and upper case "F" for false. The double quotes are important because we will compose your answers to form strings.

In [ ]:
# Replace the X with an upper case T or F
Answer1 = "X"
Answer2 = "X"

###
### YOUR CODE HERE
###

In [ ]:
#= Friendly Check =#

# Correct answers
CorrectAnswers = ["T", "F"]
YourAnswers = [Answer1, Answer2]

# Count how many are correct
num_correct_2 = sum(YourAnswers .== CorrectAnswers)

println("You got $num_correct out of 2 correct.")

In [ ]:
# ====================================================
# 🔒 GRADER CELL — Problem 2 (Do Not Edit)
# ====================================================
# This cell required previous Friendly checks to be run first.
# Please run this cell before going to the next problem to avoid variable errors.
# SAMPLE SOLUTION ==> check the CorrectAnswers variable in the friendly check.

total_score_2 = num_correct_2 / 2

# Show score
println("Problem 2 Score: $total_score_2 / 1")



<p align="center">
  <img src="data/HW03TrapezoidalRuleImage.png", width="70%">
</p>

<br>
<br>

# Problem 3: The Trapezoidal Rule


<br>

<p align="center">
  <img src="data/HW03TrapezoidalRuleStatement.png", width="85%">
</p>

<br>

## P.3A: Build a function to implement the trapezoidal rule


In [ ]:
function trapezoidalRule(f, a, b, N, aTol=1e-6)
    # Works for b < a and for b >= a
    if abs(b - a) < aTol
        println("a and b are too close together for most numerical integration methods.")
        return NaN
    end
    N = floor(Int64, N) # make sure N is an integer
    if N < 2
        N = 2
    end

    ###
    ### YOUR CODE HERE
    ###

    return myIntTrapZ
end

In [ ]:
#= Friendly Check =#
# if the value of is_it_correct_checkN is "Yes", then your answer is LIKELY correct. 
# If the value of is_it_correct_checkN is "No", then your answer is DEFINITELY wrong
#
g(x) = -2 * x^7 + x^4 - 2x^2 - 3x - 4
a, b = (-1.0, 1.0)

# compute data
myIntTrapZ01 = trapezoidalRule(g, a, b, 4)
myIntTrapZ02 = trapezoidalRule(g, a, b, 1000)

# Run Tests
is_it_correct_check3a_1 = abs(myIntTrapZ01 + 8.9375) < 1e-4 ? "Yes" : "No"
is_it_correct_check3a_2 = abs(myIntTrapZ02 + 8.93333) < 1e-4 ? "Yes" : "No"
@show is_it_correct_check3a_1;
@show is_it_correct_check3a_2;

## P.3B: Compare the trapezoidal rule to integration via rectangles

In [ ]:
# Run this cell. It creates functions and sets integration bounds and number of intervals
f(x) = tan(x^2) + x / 2
g(x) = csc(x^3) + x / 2
h(x) = exp(x / 4) - 3

a, b = (-3.0, 4.0)
N = 1e3;

include("data/basicRiemannIntegrator.jl")
# Pulls in the function myRiemannIntegralValueAndError

#Example
myEstimatedIntegral, myPMerror = myRiemannIntegralValueAndError(f, a, b, N)
myIntTrapZ = trapezoidalRule(f, a, b, N)

[myEstimatedIntegral myIntTrapZ]

Questions B.1–B.3: Comparing `myEstimatedIntegral` and `myIntTrapZ`

For each function below, choose the correct relationship between `myEstimatedIntegral` and `myIntTrapZ`.

**Question B.1: For the function `f(x)`**
- A) `myEstimatedIntegral == myIntTrapZ`  
- B) `myEstimatedIntegral > myIntTrapZ`  
- C) `myEstimatedIntegral < myIntTrapZ`  

**Question B.2: For the function `g(x)`** 
- A) `myEstimatedIntegral == myIntTrapZ`  
- B) `myEstimatedIntegral > myIntTrapZ`  
- C) `myEstimatedIntegral < myIntTrapZ`  

**Question B.3: For the function `h(x)`** 
- A) `myEstimatedIntegral == myIntTrapZ`  
- B) `myEstimatedIntegral > myIntTrapZ`  
- C) `myEstimatedIntegral < myIntTrapZ`  

**Question B.4: Relationship Between the Trapezoidal Rule and Riemann Sums**

For the same number of intervals \( N \), the Trapezoidal Rule and the average of the Riemann Lower and Upper Sums produce the same result because:

- A) The average of the maximum and minimum of two function values is equal to their arithmetic mean.  
- B) This is only true for small \( N \); as \( N \) increases, Riemann sums diverge from the trapezoidal estimate.  
- C) It’s a coincidence specific to the chosen examples. In general, overshooting and undershooting behave differently — you’ve been tricked!  
- D) All numerical integration methods are approximations. As long as they use the same number of intervals, they’ll give the same result.  
- E) None of the above are true.


### Instructions

In Questions B.1–B.3, select **A**, **B**, or **C**.  
In Question B.4, select **A**, **B**, **C**, **D**, or **E**.


In [ ]:
Answer1 = "X"
Answer2 = "X"
Answer3 = "X"
Answer4 = "X"

###
### YOUR CODE HERE
###

In [ ]:
#= Friendly Check =#

# Correct answers
CorrectAnswers = ["A", "B", "C", "A"]
YourAnswers = [Answer1, Answer2, Answer3, Answer4]

# Count how many are correct
num_correct_3 = sum(YourAnswers .== CorrectAnswers)

println("You got $num_correct_3 out of 4 correct.")

In [ ]:
# ========== PROBLEM 3 SAMPLE SOLUTION ==========
# You can use this as a reference to check your answer.

### 3A BEGIN SOLUTION
function trapezoidalRule(f, a, b, N, aTol=1e-6)
    if abs(b - a) < aTol
        println("a and b are too close together for most numerical integration methods.")
        return NaN
    end
    N = floor(Int64, N)
    if N < 2
        N = 2
    end
    Δx = (b - a) / N
    x = range(a, b, N + 1)
    y = f.(x)
    myIntTrapZ = 0.0
    for i = 1:N
        myIntTrapZ += Δx * (y[i] + y[i+1]) / 2.0
    end
    return myIntTrapZ
end
### 3A END SOLUTION

### 3B BEGIN SOLUTION
Answer1, Answer2, Answer3, Answer4 = "A", "B", "C", "A"
### 3B END SOLUTION

In [ ]:
# ====================================================
# 🔒 GRADER CELL — Problem 3 (Do Not Edit)
# ====================================================
# This cell required previous Friendly checks to be run first.
# Please run this cell before going to the next problem to avoid variable errors.

point_3A = is_it_correct_check3a_1 == "Yes" && is_it_correct_check3a_2 == "Yes" ? 1 : 0
point_3B = num_correct_3 / 2
total_score_3 = point_3A + point_3B

# Show score
println("Problem 3 Score: $total_score_3 / 3")

# Problem 4: Achieving a Prescribed Level of Precision with a Numerical Integrator

In robotics and engineering, precision is often critical. When we want to meet a specific accuracy threshold, how can we achieve that? Well, let's consider the **`pmError`** from our Riemann Lower and Upper Sums. This value gives us a measure of how precise our integral estimate is. 

Assume a uniform partition 
$a = x_1, x_2, \ldots, x_n, x_{n+1}=b$, with $x_{i+1}-x_i = \Delta x := \frac{b-a}{n}$. 

The Upper and Lower Sums are defined by:

$$
\begin{align*}
{\rm Area}^{up}\_n &:= \sum_{i=1}^n {\rm max} \{ f(x_i), f(x_{i+1}) \} \Delta x \\\\
{\rm Area}^{low}\_n &:= \sum_{i=1}^n {\rm min} \{ f(x_i), f(x_{i+1}) \} \Delta x
\end{align*}
$$

From these, we define the **total precision error**:


$$ 2 \cdot {\rm pmError}\_n:= {\rm Area}^{up}\_n - {\rm Area}^{low}\_n =  \sum_{i=1}^n \left( {\rm max}\\{f(x_i), f(x_{i+1}) \\} - {\rm min}\\{f(x_i), f(x_{i+1}) \\} \right) \Delta x  = \sum_{i=1}^n |f(x_i) - f(x_{i+1})| \Delta x$$


This simplification is valid because:

$$ \left( {\rm max}\\{f(x_i), f(x_{i+1}) \\} - {\rm min}\\{f(x_i), f(x_{i+1}) \\}\right) = |f(x_i) - f(x_{i+1})|.$$

<br>

Therefore, we can estimate the precision error associated with the Trapezoidal Rule by computing: $${\rm pmError}\_n = \sum_{i=1}^n \frac{1}{2}|f(x_i) -f(x_{i+1})| \Delta x.$$


<br>


## P.4A: Adding an error estimate

In [ ]:
# Copy in any useful code from trapezoidalRule(f, a, b, N, aTol=1e-6) and add in the  
# additional two lines required to compute the error estimate. 
# We have specified the return for you.

function trapezoidalRulewithpmError(f, a, b, N, aTol=1e-6)
    # Works for b < a and for b >= a
    if abs(b - a) < aTol
        println("a and b are too close together for most numerical integration methods.")
        return NaN
    end
    N = floor(Int64, N) # make sure N is an integer
    if N < 2
        N = 2
    end
    # Copy in code from your previous function, trapZ, then add  
    # in the extra lines to initialize and sum the terms in pmError    

    ###
    ### YOUR CODE HERE
    ###

    return myIntTrapZ, pmError
end

In [ ]:
#= Friendly Check =#
# if the value of is_it_correct_checkN is "Yes", then your answer is LIKELY correct. 
# If the value of is_it_correct_checkN is "No", then your answer is DEFINITELY wrong
#
g(x) = -2 * x^7 + x^4 - 2x^2 - 3x - 4
a, b = (-1.0, 1.0)

# compute data
myIntTrapZ02, pmError02 = trapezoidalRulewithpmError(g, a, b, 1000)

# Run Tests
is_it_correct_check4a_1 = abs(myIntTrapZ02 + 8.93333) < 1e-4 ? "Yes" : "No"
is_it_correct_check4a_2 = abs(pmError02 - 0.01) < 1e-4 ? "Yes" : "No"
@show is_it_correct_check4a_1;
@show is_it_correct_check4a_2;

if (is_it_correct_check4a_1 == "Yes") && (is_it_correct_check4a_2 == "Yes")
    println("\nCongrats! You now have a means to estimate the accuracy of the Trapezoidal Rule.")
end

## P.4B: Adaptive integration using `pmError`
In this task, you'll implement an adaptive version of the Trapezoidal Rule.

Just like in the Bisection Algorithm, you'll repeatedly compute the integral using increasingly larger values of \( N \) (the number of trapezoids) until the estimated error `pmError` falls below a specified tolerance.

For the tolerance, use the parameter `aTol` that you've already defined.

In [ ]:
# Copy in any useful code from trapezoidalRulewithpmError(f, a, b, N, aTol=1e-6) and add in the additional 
# two lines required to compute the error estimate

function trapezoidalRuleFixedTolerance(f, a, b, aTol=1e-5)
    # Works for b < a and for b >= a
    if abs(b - a) < aTol
        println("a and b are too close together for most numerical integration methods.")
        return NaN
    end
    N = 100 # Initialize at a small number of trapezoids
    Nmax = 1e8 # Prevent infinite loop
    pmError = aTol + 1.0 # make sure it starts larger than the tolerance
    myIntTrapZ = 0.0 # to return it, we must initialize the variable before the while loop
    while (pmError > aTol) && (N <= Nmax)
        N = 10 * N # Increase N by a fixed factor each time
        #    

        ###
        ### YOUR CODE HERE
        ###

    end
    return myIntTrapZ, pmError, N
end

In [ ]:
#= Friendly Check =#
# if the value of is_it_correct_checkN is "Yes", then your answer is LIKELY correct. 
# If the value of is_it_correct_checkN is "No", then your answer is DEFINITELY wrong
#

f(x) = x^7 + x^5 + 2.0 * x
a, b = (-1, 1)

myIntTrapZ, pmError, N = trapezoidalRuleFixedTolerance(f, a, b)

# Run Tests
is_it_correct_check4b_1 = abs(myIntTrapZ) < 1e-6 ? "Yes" : "No"
is_it_correct_check4b_2 = pmError < 1e-5 ? "Yes" : "No"
is_it_correct_check4b_3 = 2e5 < N < 2e6 ? "Yes" : "No"
@show is_it_correct_check4b_1;
@show is_it_correct_check4b_2;
@show is_it_correct_check4b_3;

if (is_it_correct_check4b_1 == "Yes") && (is_it_correct_check4b_2 == "Yes") && (is_it_correct_check4b_3 == "Yes")
    println("\nCongrats! You can now numerically integrate anything, and 
          have confidence in the answer!")
end

In [ ]:
# ========== PROBLEM 4 SAMPLE SOLUTION ==========
# You can use this as a reference to check your answer.

### 4A BEGIN SOLUTION
function trapezoidalRulewithpmError(f, a, b, N, aTol=1e-6)
    if abs(b - a) < aTol
        println("a and b are too close together for most numerical integration methods.")
        return NaN
    end
    N = floor(Int64, N)
    if N < 2
        N = 2
    end
    Δx = (b - a) / N
    x = range(a, b, N + 1)
    y = f.(x)
    myIntTrapZ = 0.0
    pmError = 0.0
    for i = 1:N
        myIntTrapZ += Δx * (y[i] + y[i+1]) / 2.0
        pmError += Δx * abs(y[i] - y[i+1]) / 2.0
    end
    return myIntTrapZ, pmError
end
### 4A END SOLUTION

### 4B BEGIN SOLUTION
function trapezoidalRuleFixedTolerance(f, a, b, aTol=1e-5)
    if abs(b - a) < aTol
        println("a and b are too close together for most numerical integration methods.")
        return NaN
    end
    N = 100
    Nmax = 1e8
    pmError = aTol + 1.0
    myIntTrapZ = 0.0
    while (pmError > aTol) && (N <= Nmax)
        N = 10 * N
        Δx = (b - a) / N
        x = range(a, b, N + 1)
        y = f.(x)
        myIntTrapZ = 0.0
        pmError = 0.0
        for i = 1:N
            myIntTrapZ += Δx * (y[i] + y[i+1]) / 2.0
            pmError += Δx * abs(y[i] - y[i+1]) / 2.0
        end
    end
    return myIntTrapZ, pmError, N
end
### 4B END SOLUTION

In [ ]:
# ====================================================
# 🔒 GRADER CELL — Problem 4 (Do Not Edit)
# ====================================================
# This cell required previous Friendly checks to be run first.
# Please run this cell before going to the next problem to avoid variable errors.
is_it_correct_check4a = (is_it_correct_check4a_1 == "Yes") && (is_it_correct_check4a_2 == "Yes")
is_it_correct_check4b = (is_it_correct_check4b_1 == "Yes") && (is_it_correct_check4b_2 == "Yes") && (is_it_correct_check4b_3 == "Yes")

f(x) = x^3 + 2x^2 + 3x + 4
a, b = (0, 1)
myIntTrapZ1, pmError1 = trapezoidalRulewithpmError(f, a, b, 1000)
myIntTrapZ2, pmError2, N = trapezoidalRuleFixedTolerance(f, a, b)

point_4A = is_it_correct_check4a && abs(myIntTrapZ1 - 6.41666) < 1e-3 && abs(pmError1 - 0.00299) < 1e-3 ? 1 : 0
point_4B = is_it_correct_check4b && abs(myIntTrapZ2 - 6.41666) < 1e-3 && abs(pmError2) < 1e-5 && (2e5 < N < 2e6) ? 1 : 0
total_score_4 = point_4A + point_4B

# Show score
println("Problem 4 Score: $total_score_4 / 2")

# Problem 5: Simpson's Rule

- We'll provide code for the **cubic version** of Simpson's Rule.
- Your task is to implement the **standard quadratic version** as described in the textbook.

In [ ]:
function cubicSimpsonRule(f, a, b, N, aTol=1e-6)
    # Works for b < a and for b >= a
    if abs(b - a) < aTol
        println("a and b are too close together for most numerical integration methods.")
        return NaN
    end
    N = floor(Int64, N) # make sure N is an integer
    if N < 2
        N = 2
    end
    Δx = (b - a) / N
    h = Δx
    x = range(a, b, N + 1)
    myIntCubSimp = 0.0
    for i = 1:N
        myIntCubSimp += (h / 8.0) * (f(x[i]) + 3 * f(x[i] + h / 3.0) + 3 * f(x[i] + 2.0 * h / 3.0) + f(x[i+1]))
    end
    return myIntCubSimp
end

In [ ]:
# Run me to see how the function does
# Re-run your function exactPolynomialIntegrator(polyCoeff, a, b) 
# to make it active again
f(x) = x^8 - 20.0 * x^7 + x^6 + x^4 + 2.0 * x^3 + 1
a, b = (-5, 5)
N = 1e2

polyCoeff = [1; -20.0; 1; 0.0; 1; 2; 0; 0; 1]
intAtoB = exactPolynomialIntegrator(polyCoeff, a, b)

myIntCubSimp = cubicSimpsonRule(f, a, b, N)

println("Looks pretty good! The error is $(myIntCubSimp - intAtoB) out of $intAtoB\n")
[intAtoB; myIntCubSimp]

### Note

Between each pair of points, $[x_i, x_{i+1}]$, the cubic version of Simpson’s Rule inserts two additional points: $x_i + \frac{\Delta x}{3}$ and $x_i + 2\frac{\Delta x}{3}$. This gives it three times as many points as the Trapezoidal Rule. Let’s keep that in mind as we compare the methods using a polynomial, where the exact integral is known. Remember, we rely on numerical integration because for most functions, a closed-form expression for the definite integral doesn’t exist.

In [ ]:
f(x) = x^8 - 20 * x^7 + x^6 + x^4 + 2.0 * x^3 + 1
a, b = (-5, 5)
N = 1e2

polyCoeff = [1; -20; 1; 0.0; 1; 2; 0; 0; 1]
intAtoB = exactPolynomialIntegrator(polyCoeff, a, b)

myIntCubSimp = cubicSimpsonRule(f, a, b, N)

myIntTrapZ = trapezoidalRule(f, a, b, 3 * N)

println("The error in the Trapezoidal Rule is $(myIntTrapZ - intAtoB) out of $intAtoB using $(3N) trapezoids.")
println("Hence, in a pinch, all of these methods are viable for estimating the integrals of complicated functions.\n")
[intAtoB; myIntCubSimp; myIntTrapZ]

For your final numerical integration task, implement the standard (quadratic) version of Simpson’s Rule as described in the textbook.

In [ ]:
function stdSimpsonRule(f, a, b, N, aTol=1e-6)

    ###
    ### YOUR CODE HERE
    ###

    return myIntSimp
end

In [ ]:
#= Friendly Check =#
# if the value of is_it_correct_checkN is "Yes", then your answer is LIKELY correct.
# If the value of is_it_correct_checkN is "No", then your answer is DEFINITELY wrong

f(x) = x^8 - 20.0 * x^7 + x^6 + x^4 + 2.0 * x^3 + 1
a, b = (-5, 5)
N = 1e2

myIntSimp = stdSimpsonRule(f, a, b, N)

is_it_correct_check5_1 = abs(myIntSimp - 457609.28) < 1e-1 ? "Yes" : "No"

@show is_it_correct_check5_1

In [ ]:
# ========== PROBLEM 5 SAMPLE SOLUTION ==========
# You can use this as a reference to check your answer.

### 5 BEGIN SOLUTION
function stdSimpsonRule(f, a, b, N, aTol=1e-6)
    if abs(b-a) < aTol
        println("a and b are too close together for most numerical integration methods.")
        return NaN
    end
    N = floor(Int64, N)
    if N < 2
        N = 2
    end 
    Δx = (b-a)/N
    h = Δx
    x = range(a, b, N+1)   
    myIntSimp = 0.0
    for i = 1:N        
        myIntSimp += (h/6.0) * ( f(x[i]) + 4*f(x[i] + h/2.0) + f(x[i+1]) )
    end   
    return myIntSimp
end
### 5 END SOLUTION

In [ ]:
# ====================================================
# 🔒 GRADER CELL — Problem 5 (Do Not Edit)
# ====================================================
# This cell required previous Friendly checks to be run first.
# Please run this cell before going to the next problem to avoid variable errors.

polyCoeff = [1; -20; 1; 0.0; 1; 2; 0; 0; 1]
intAtoB = exactPolynomialIntegrator(polyCoeff, a, b)
myIntCubSimp = cubicSimpsonRule(f, a, b, N)

is_it_correct_check5_2 = (intAtoB < myIntCubSimp < myIntSimp)

point_5 = is_it_correct_check5_1 == "Yes" && is_it_correct_check5_2 ? 1 : 0
total_score_5 = point_5

# Show score
println("Problem 5 Score: $total_score_5 / 1")

# Problem 6: Generalized Additivity


<br>

<p align="center">
  <img src="data/HW03GenearlizedAdditivityProperty.png", width="70%">
</p>

<br>

For each of the following sums of integrals, rewrite the expression as a single integral that has the same value. Your task is to identify the appropriate lower limit of integration, $a_k$, and upper limit of integration, $b_k$.

### Note
You can check your answers with:
```julia
f(x) = sin(x)/(1 + x^2)
myAns = cubicSimpsonRule(f, a, b, 20)
```

## P.6A

$$\int_1^4 f(x) dx + \int_{4}^{8} f(x) dx = \int_{a_1}^{b_1} f(x) dx$$

In [ ]:
# The same value is obtained with
#a1 = ?
#b1 = ?

###
### YOUR CODE HERE
###

In [ ]:
#= Friendly Check =#
f(x) = sin(x) / (1 + x^2)
myFirstIntegral = cubicSimpsonRule(f, 1, 4, 20)
mySecondIntegral = cubicSimpsonRule(f, 4, 8, 20)

myEquivalentIntegral = cubicSimpsonRule(f, a1, b1, 20)

is_it_correct_check6a = (abs(myEquivalentIntegral - myFirstIntegral - mySecondIntegral) < 1e-2)
if is_it_correct_check6a
    println("I am so good at Calculus, I should become a math major!")
else
    println("Ooops, I messed up somewhere. I hate it when that happens!")
end

## P.6B
$$\int_{10}^6 f(x) dx + \int_{1}^{10} f(x) dx = \int_{a_2}^{b_2} f(x) dx$$


In [ ]:
# The same value is obtained with 
#a2 = ?
#b2 = ?

###
### YOUR CODE HERE
###

In [ ]:
#= Friendly Check =#
myEquivalentIntegral = cubicSimpsonRule(f, a2, b2, 20)

is_it_correct_check6b = abs(myEquivalentIntegral - 0.30390311635917966) < 1e-2 ? "Yes" : "No"

@show is_it_correct_check6b

## P.6C
$$\int_{-1}^3 f(x) dx + \int_{5}^{11} f(x) dx +  \int_{11}^{-1} f(x) dx= \int_{a_3}^{b_3} f(x) dx$$

**Hint:** Simplify two of the integrals and then bring in the third integral.

In [ ]:
# The same value is obtained with 
#a3 = ?
#b3 = ?

###
### YOUR CODE HERE
###

In [ ]:
#= Friendly Check =#
myEquivalentIntegral = cubicSimpsonRule(f, a3, b3, 20)

is_it_correct_check6c = abs(myEquivalentIntegral - 0.06695099457741466) < 1e-2 ? "Yes" : "No"

@show is_it_correct_check6c

In [ ]:
# ========== PROBLEM 6 SAMPLE SOLUTION ==========
# You can use this as a reference to check your answer.

### 6A BEGIN SOLUTION
a1, b1 = (1, 8)
### 6A END SOLUTION

### 6B BEGIN SOLUTION
a2, b2 = (1, 6)
### 6B END SOLUTION

### 6C BEGIN SOLUTION
a3, b3 = (5, 3)
### 6C END SOLUTION

In [ ]:
# ====================================================
# 🔒 GRADER CELL — Problem 6 (Do Not Edit)
# ====================================================
# This cell required previous Friendly checks to be run first.
# Please run this cell before going to the next problem to avoid variable errors.

point_6A = is_it_correct_check6a ? 1 : 0
point_6B = is_it_correct_check6b == "Yes" ? 1 : 0
point_6C = is_it_correct_check6c == "Yes" ? 1 : 0
total_score_6 = (point_6A + point_6B + point_6C) / 2

# Show score
println("Problem 6 Score: $total_score_6 / 1.5")

In [ ]:
# ======= Final Score Summary ======= #
# You have to run all previous grader cells to get the final score summary.
total_score = total_score_1 + total_score_2 + total_score_3 + total_score_4 + total_score_5 + total_score_6

println("🎉🎯 Final Score Summary 🎯🎉")
println("────────────────────────────")
println("Problem 1: $total_score_1")
println("Problem 2: $total_score_2")
println("Problem 3: $total_score_3")
println("Problem 4: $total_score_4")
println("Problem 5: $total_score_5")
println("Problem 6: $total_score_6")
println("────────────────────────────")
println("🏆 Total Score: $total_score / 10.5")


<p align="center" style="font-size:48px;"><strong>The End! </strong></p>




